In [66]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

# Setup

In [67]:
# Get source file
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read()
print('corpus length:', len(text))

corpus length: 600901


In [68]:
# Create a vocabulary of chars in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 86


In [69]:
chars.insert(0, "\0")

In [70]:
''.join(chars[1:-6])

'\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz'

In [71]:
char_indices = dict((c,i) for i,c in enumerate(chars))
indices_char = dict(enumerate(chars))

In [72]:
# convert whole text into (char)indices, a new representation
idx = [char_indices[c] for c in text]

In [73]:
idx[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

In [74]:
''.join(indices_char[i] for i in idx[:70])

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not gro'

In [75]:
cs=3
c1_dat = [idx[i]   for i in xrange(0, len(idx)-1-cs, cs)] #0, 3, 6, ... 
c2_dat = [idx[i+1] for i in xrange(0, len(idx)-1-cs, cs)] #1, 4, 7, ...
c3_dat = [idx[i+2] for i in xrange(0, len(idx)-1-cs, cs)] #2, 5, 8, ...
c4_dat = [idx[i+3] for i in xrange(0, len(idx)-1-cs, cs)] #3, 6, 9, ...

In [76]:
x1 = np.stack(c1_dat[:-2])
x2 = np.stack(c2_dat[:-2])
x3 = np.stack(c3_dat[:-2])

In [77]:
y = np.stack(c4_dat[:-2])

In [78]:
x1[:4], x2[:4], x3[:4]

(array([40, 30, 29,  1]), array([42, 25,  1, 43]), array([29, 27,  1, 45]))

In [79]:
y[:3]

array([30, 29,  1])

In [80]:
x1.shape, y.shape

((200297,), (200297,))

In [81]:
n_fac = 42

In [82]:
def embedding_input(name, n_in, n_out):
    inp = Input (shape=(1,), dtype='int64', name=name)
    emb = Embedding(n_in, n_out, input_length=1)(inp)
    return inp, Flatten()(emb)

In [83]:
c1_in, c1 = embedding_input('c1', vocab_size, n_fac)
c2_in, c2 = embedding_input('c2', vocab_size, n_fac)
c3_in, c3 = embedding_input('c3', vocab_size, n_fac)

In [84]:
type(c1_in)

theano.tensor.var.TensorVariable

# Model

In [88]:
# Number of neurons in the hidden layer
n_hidden = 256

In [86]:
# Create a skeleton of a dense hidden layer w/ n_hidden neurons and relu activation
# Partial function
dense_in = Dense(n_hidden, activation='relu')

In [87]:
# Create the dense layer w/ the c1 embeddings
c1_hidden = dense_in(c1)

In [89]:
dense_hidden = Dense(n_hidden, activation='tanh')

In [91]:
c2_dense = dense_in(c2) # Create a second layer embedding
hidden_2 = dense_hidden(c1_hidden) #
c2_hidden = merge([c2_dense, hidden_2])

In [92]:
c3_dense = dense_in(c3) # Create a second layer embedding
hidden_3 = dense_hidden(c2_hidden) #
c3_hidden = merge([c3_dense, hidden_3])

In [93]:
dense_out = Dense(vocab_size, activation='softmax')

In [94]:
c4_out = dense_out(c3_hidden)

In [96]:
model = Model([c1_in, c2_in, c3_in], c4_out)

In [97]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [98]:
model.optimizer.lr = 0.001

In [99]:
model.fit([x1,x2,x3], y, batch_size=64, nb_epoch=1)

Epoch 1/1
200297/200297 [==============================] - 14s - loss: 2.3988      

In [101]:
np.array(2)[np.newaxis]

array([2])

In [103]:
np.array([1,2,3])

array([1, 2, 3])

In [104]:
def get_next(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict(arrs)
    i = np.argmax(p)
    return chars[i]

In [107]:
get_next('phi')

's'

In [108]:
get_next(' th')

'e'

In [109]:
get_next(' an')

'd'

In [56]:
!git add .

In [57]:
!git commit -m "Add basic Python notebooks and setup"

[master f6490d4] Add basic Python notebooks and setup
 5 files changed, 864 insertions(+)
 create mode 100644 .gitignore
 create mode 100644 rnn.ipynb
 create mode 100755 utils.py
 create mode 100755 vgg16.py
 create mode 100755 vgg16bn.py


In [58]:
!git push -u origin master

Counting objects: 7, done.
Delta compression using up to 8 threads.
Compressing objects: 100% (6/6), done.
Writing objects: 100% (7/7), 6.13 KiB | 0 bytes/s, done.
Total 7 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), done.
To github.com:szalmaf/recurrent_neural_network.git
   4e5115a..f6490d4  master -> master
Branch master set up to track remote branch master from origin.
